<a href="https://colab.research.google.com/github/nunocesarsa/Examples/blob/main/GEE_Clipper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connect to GEE

In [ ]:
#importing Earth Engine package
import ee

#then we need to authenticate
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Za4oLRMDHERHI27hiwtrE5LRnBTDx-uy_k_pq6DZsMA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6viErytlRL47bUwFGw855QIujNtWMAn1pd7Lxpy2Gp6u3J4POBWYM

Successfully saved authorization token.


#Mounting Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Parameters

In [ ]:
#AOI to crop:
poly_area_coord = [[[4.26291867530725,52.08374234863279],
                    [4.450716221693969,52.08374234863279],
                    [4.450716221693969,52.208254433484235],
                    [4.26291867530725,52.208254433484235],
                    [4.26291867530725,52.08374234863279]]]

#point to guide image search filter 
point_coord = [4.3177644852437735,52.12475518385005]


#creating objects:

#This creates an earth engine polygon geometry
poly_area = ee.Geometry.Polygon(poly_area_coord)
point_geo = ee.Geometry.Point(point_coord)


##Search collection

In [ ]:
#check the filters
s2_collection = (ee.ImageCollection("COPERNICUS/S2_SR") #selects the L8 Surface reflectance product in GEE
                 .select(['B2','B3','B4','B8'
                          #'B5','B6','B7',
                          #'B8A','B11','B12'
                          ]) #selects the bands of interest
                 .filter(ee.Filter.date('2018-06-01','2018-10-30')) #filters on the time - you can play around with this to select different dates..
                 .filterBounds(point_geo) #filters on the Aoi we have created above
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)))

count = s2_collection.size()
print('Nr of images in collection: ', str(count.getInfo())+'\n')

Nr of images in collection:  1



### Exploring metadata of ith image (optional)

In [ ]:
mid_step = s2_collection.toList(count)
s2_img = ee.Image(mid_step.get(0))
s2_img_meta = s2_img.getInfo()
s2_img_meta

{'bands': [{'crs': 'EPSG:32631',
   'crs_transform': [10, 0, 499980, 0, -10, 5800020],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [10980, 10980],
   'id': 'B2'},
  {'crs': 'EPSG:32631',
   'crs_transform': [10, 0, 499980, 0, -10, 5800020],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [10980, 10980],
   'id': 'B3'},
  {'crs': 'EPSG:32631',
   'crs_transform': [10, 0, 499980, 0, -10, 5800020],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [10980, 10980],
   'id': 'B4'},
  {'crs': 'EPSG:32631',
   'crs_transform': [10, 0, 499980, 0, -10, 5800020],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [10980, 10980],
   'id': 'B8'}],
 'id': 'COPERNICUS/S2_SR/20180630T105031_20180630T105440_T31UET',
 'properties': {'AOT_RETRIEVAL_ACCUR

## Exploring all the images in the dataset 

In [ ]:
for i in range(count.getInfo()):
  mid_step = s2_collection.toList(count) #this is an internal step to help us loop on the data itself
  s2img = ee.Image(mid_step.get(i)) #this fetches ith image on the collection 
  s2_img_meta = s2_img.getInfo()   #this creates an object which has only the metadata 
  print('At position',i,'is Product ID: ',s2_img_meta.get('properties',{}).get('PRODUCT_ID'))

  #interpreting the output:
  #Landsat naming convention:
  #XXXX_YYYYYY_ZZZZZZZZ
  #XXXX <- Product identifier LC08 is Landsat 8, level C
  #YYYYYYY <- Path and row identifers -> USGS pushes the LS images into a grid for the entire planet, this is the idenfier of said grid
  #ZZZZZZZZ <- Date in western calendar

At position 0 is Product ID:  S2A_MSIL2A_20170923T105021_N0205_R051_T31UET_20170923T105022


#Exploring the image 

For this, we import folium pacakge, and create a specific function to assist. This is to make a prettier and more explorable map than the normal.

(this was copy pasted, probably there are other ways of doing the same).

This tutorial gives an excellent run down example of the many options in folium

https://spatial.utk.edu/maps/ee-api-folium-setup.html

In [ ]:
#in here, we select an image of the list - in python lists start at 0
mid_step = s2_collection.toList(count)
s2_img = ee.Image(mid_step.get(0))

#this command clips the image based on the original poly area
s2_img_clip = s2_img.clip(poly_area)

# Import the Folium library. This library is to assist in visualizations
import folium

# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Define a method for displaying Earth Engine image tiles to folium map.
    #the next code snippet i stole from someone
    #this is a simplified version of the original function
def add_ee_layer(self, ee_image_object, vis_params, name):
  
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)



# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#starting location of the map
my_map = folium.Map(location=[52.1326,5.2913], zoom_start=8) #this had to be tried one by one... feels daft

# Set visualization parameters. Here is we can create different visualizations of our data
visParams = {"bands": ['B4', 'B3', 'B2'],"gain": '0.1, 0.1, 0.1',"scale":20}
visParams_nir = {"bands": ['B8', 'B4', 'B3'],"gain": '0.1, 0.1, 0.1',"scale":20}

# Add the image to the folium map
my_map.add_ee_layer(s2_img, visParams, 'RGB composite')
my_map.add_ee_layer(s2_img_clip, visParams_nir, 'NIR composite - clipped')
my_map.add_ee_layer(s2_img, visParams_nir, 'NIR composite')

#adding a layer control
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

#Downloading a selection

Now, here, read carefully. The way it works is the google colab will send a "task" order to GEE and that task will be running on the background.

This means that if you run the next box many times you will have multiple tasks being sent.

On the other hand, if you save a "big image" then you take a lot of time and space of your drive. So you can just extract the clipped area of your study location and focus on that - makes it easier to set up your own models on your own machine this way.

In [ ]:
#getting the filenames
imagename = s2_img_meta.get('properties',{}).get('PRODUCT_ID')

s2_img_meta.get('properties',{}).get('PRODUCT_ID')

'S2A_MSIL2A_20180630T105031_N0208_R051_T31UET_20180630T144133'

##Tasking GEE

In [ ]:
#these are task commands to be sent to GEE
task_config = {
    'image': s2_img,
    'fileFormat': 'GeoTIFF',
    'folder': 'Meijendel', #Meijendel folder 
    'fileNamePrefix': imagename[0:19] + "_Meijendel",
    'description': "clipped area",
    'scale':10,
    'region':poly_area
}


In [ ]:
#This is how we order it to start
task = ee.batch.Export.image.toDrive(**task_config )
task.start()

##Check task status

In [ ]:
#to check if the task is running or completed, or failed, run this
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1619682943973,
 'description': 'clipped area',
 'id': 'U4PTHUGAAXSKR2W5E22MAU2B',
 'name': 'projects/earthengine-legacy/operations/U4PTHUGAAXSKR2W5E22MAU2B',
 'start_timestamp_ms': 1619682957376,
 'state': 'RUNNING',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1619682960765}